In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib
import sqlite3
##########모델 로드

model = tf.keras.models.load_model('C:\\Anomaly_Detection_paper_review.h5')

In [2]:
def db_to_df(loc): 
    #df3=pd.DataFrame
    conn =sqlite3.connect(loc)
    c=conn.cursor()
    c.execute('SELECT srcSpeed,srcAPS,srcGyroValue,srcRPM,srcTPS,srcMAF,srcEngineLoad FROM SRCREC')
    df3 = pd.read_sql('SELECT srcSpeed,srcAPS,srcGyroValue,srcRPM,srcTPS,srcMAF,srcEngineLoad FROM SRCREC', conn, index_col=None)
    conn.close()
    return df3

In [3]:
def df_60s_mean(input1,length):
    df1=pd.DataFrame(columns=['srcSpeed','srcAPS','srcGyroValue','srcRPM','srcTPS','srcMAF','srcEngineLoad'])
    rest=input1.shape[0]%60
    quotient = input1.shape[0]//60
    if quotient>0:
        for i in range(0,quotient):
            df_temp=(input1.iloc[60*i:60*(i+1),:].mean()).to_frame().transpose()
            df1=df1.append(df_temp, ignore_index = True)
        df_temp=(input1.iloc[quotient*60:,:].mean()).to_frame().transpose()
        df1=df1.append(df_temp, ignore_index = True)
    else :
        df_temp=(input1.mean()).to_frame().transpose()
        df1=df1.append(df_temp, ignore_index = True)

    return df1

In [4]:
def anomaly_count(input2):
    x_test1 = input2
    # normalize the data
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(x_test1)
    x_test = scaler.transform(x_test1)
    scaler_filename = "scaler_data"
    joblib.dump(scaler, scaler_filename)

    # reshape inputs for LSTM [samples, timesteps, features]
    x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1])
    print("Test data shape:", x_test.shape)

    scored = pd.DataFrame(index=x_test1.index)
    X_pred = model.predict(x_test)
    X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
    X_pred = pd.DataFrame(X_pred, columns=x_test1.columns)
    X_pred.index = x_test1.index

    Xtest = x_test.reshape(x_test.shape[0], x_test.shape[2])
    scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
    scored['Threshold'] = np.percentile(scored,100)
    #(scored['Loss_mae'].mean)*2
    scored['Anomaly'] = scored['Loss_mae'] > (scored['Loss_mae'].mean())*2
    return (scored[scored['Anomaly']==True].shape[0])

In [5]:
#loc='../sample.db'
#df3=db_to_df(loc)
#df2=df_60s_mean(df3,length)
df3 = pd.read_csv('C:\\.csv',usecols=['srcSpeed','srcAPS','srcGyroValue','srcRPM','srcTPS','srcMAF','srcEngineLoad'])


In [6]:
length = df3.shape[0]
df3

,srcSpeed,srcAPS,srcGyroValue,srcRPM,srcTPS,srcMAF,srcEngineLoad
0,0.000000,15.294118,0.148298,992.158333,8.921569,0.000000,17.124183
1,0.000000,15.294118,0.017677,1109.172414,8.951995,0.000000,16.659906
2,46.133333,0.000000,-0.353861,2221.808333,25.326797,6.474333,28.000000
3,52.312500,0.000000,-0.708710,1059.421875,16.899510,3.009375,25.563725
4,0.000000,15.078431,0.000000,760.641667,13.725491,1.575000,27.764706
...,...,...,...,...,...,...,...
193,0.000000,15.294118,0.323894,266.519231,14.841628,3.985385,45.822023
194,0.000000,15.294118,-0.061867,747.433333,11.888889,3.314000,39.934641
195,0.000000,15.294118,-0.070098,753.291667,12.156863,3.770833,44.035948
196,0.000000,15.294118,0.068259,760.587500,10.274509,2.086667,24.392157


In [7]:
anomaly_count(df3)

Test data shape: (198, 1, 7)
7/7 [==============================] - 1s 2ms/step


2